## Parse cert data
TY to pyca for doing all the heavy lifting.

See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate and https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.

Pivoted away from PyCryptodome API which parses [PEM](https://github.com/Legrandin/pycryptodome/blob/fc272f6c0d85779cff67c928c798ef68f532923e/lib/Crypto/IO/PEM.py) but does not appear to parse X.509.

In [1]:
from cryptography import x509

In [2]:
with open("google-com.pem", "rb") as pem_file:
    cert = x509.load_pem_x509_certificate(pem_file.read())

## Verify signature on leaf
https://www.pycryptodome.org/src/signature/dsa

In [3]:
from Crypto.Signature import DSS
from Crypto.PublicKey import ECC

Copy & paste from output of `openssl x509 -in google-com.pem -text`.

In [4]:
pubkey = "04:83:f0:ba:89:ac:a3:38:e9:8c:5c:fb:d4:33:80:8b:13:2e:51:d1:12:94:e9:f6:97:f7:57:aa:fa:dd:9b:b5:2f:4a:a8:bd:61:bd:85:ad:04:73:3d:a3:77:40:e6:2d:20:31:ca:5c:fe:b0:db:50:c5:33:87:19:41:bd:28:0c:21"
len(pubkey)

194

https://www.pycryptodome.org/src/public_key/ecc#Crypto.PublicKey.ECC.import_key

In [5]:
key = ECC.import_key(bytes.fromhex(pubkey.replace(":", "")), curve_name="p256")
key

EccKey(curve='NIST P-256', point_x=59678313896468765492747125307157180810548982517242706764109114159404492240175, point_y=33769288167177865617267864813701956844211178309078964428519996731432488143905)

In [6]:
cert.public_key_algorithm_oid

<ObjectIdentifier(oid=1.2.840.10045.2.1, name=id-ecPublicKey)>

In [7]:
from Crypto.Hash import SHA256

See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.tbs_certificate_bytes.

In [8]:
h = SHA256.new(cert.tbs_certificate_bytes)
print(h.hexdigest(), h.oid)

d38564210b319bbcf48ed85865f88bf780cbca4794be2b8e7dccd94914cd4155 2.16.840.1.101.3.4.2.1


In [9]:
cert.signature_algorithm_oid

<ObjectIdentifier(oid=1.2.840.10045.4.3.2, name=ecdsa-with-SHA256)>

Again, copy & paste from `openssl x509` output.

In [10]:
sig_value = "30:45:02:21:00:ac:6f:77:9b:5e:3e:8d:14:93:ce:95:4f:fa:cf:61:80:8a:2c:0a:b0:3d:36:d8:f8:17:74:52:db:67:f9:08:ad:02:20:58:9d:dc:2e:a1:f5:fe:35:b8:00:c9:cc:91:ec:c9:3e:80:12:57:05:a5:37:16:8f:6a:39:9c:ea:96:09:0e:94"

In [11]:
sig = bytes.fromhex(sig_value.replace(":", ""))
len(sig)

71

In [12]:
assert sig == cert.signature # Sanity check.

More guesses. :-\

In [13]:
verifier = DSS.new(key, mode="fips-186-3", encoding="der")
#verifier = DSS.new(key, "deterministic-rfc6979", encoding="der")

In [14]:
verifier.verify(h, sig)

ValueError: The signature is not authentic